# Otter-Grader Demo Notebook



This notebook demonstrates how to use otter-grader. For installation details, see the [README](https://github.com/ucbds-infra/otter-grader).

In [48]:
import pandas as pd
import os
from subprocess import PIPE
import subprocess

## Student Usage

### Notebooks

Otter supports in-notebook checks so that students can check their progress when working through an assignments. To use it, you need to create an instance of `otter.Notebook`, to which you (optionally) pass a path to a directory of tests; this defaults to `./tests`.

In [24]:
import otter
grader = otter.Notebook()

To check a student's work, use `Notebook.check`. Pass to this the question identifier, which is the filename (without the `.py` extension). For example, in this demo, `./tests/q4.py` tests a student's `square` function:

In [2]:
with open("./tests/q4.py") as f:
    print(f.read())

test = {
	"name": "q4",
	"points": 1,
	"suites": [ 
		{
			"cases": [ 
				{
					"code": r"""
					>>> square(5)
					25
					>>> square(2.5)
					6.25
					"""
				},
			],
			"scored": False,
			"setup": "",
			"teardown": "",
			"type": "doctest"
		}, 
	]
}



If we have the square function defined below, we can run the test using `grader.check("q4")`:

In [3]:
def square(x):
    return x**2

In [4]:
grader.check("q4")

If we change the function so that the tests fail, then the grader outputs the failed test and the incorrect output.

In [5]:
square = lambda x: x**3
grader.check("q4")

Students can also run all tests at once using `Notebook.check_all`:

In [6]:
grader.check_all()

Students can also create their own PDF submissions using `Notebook.export`, to which you pass the path to the notebook. Otter uses nb2pdf to generate PDFs (more info [below](#nb2pdf)). The filtering behavior of nb2pdf is turned on by default, although it can be turned off using the `filtering=False` argument.

In [7]:
grader.export("demo.ipynb")

### Scripts

If a student is working with Python scripts instead of notebooks, the otter command line utility has a `check` command that can run these tests as well. The help entry for this command is given below.

In [32]:
!otter check -h

usage: otter [-h] [-q QUESTION] [-t TESTS-PATH] file

positional arguments:
  file           Python file to grade

optional arguments:
  -h, --help     show this help message and exit
  -q QUESTION    Grade a specific test
  -t TESTS-PATH  Path to test files


If we wanted to run a single check, we would pass the question identifier (the filename without `.py`) to the `-q` flag. The tests path is assumed to be `./tests` unless another path is provided to the `-t` flag.

In [35]:
!otter check scripts/file0.py -q q4

All tests passed!


As with notebooks, a failed test will output the test and the incorrect output.

In [36]:
!otter check scripts/file0.py -q q2

1 of 2 tests passed

Tests passed:
 possible 


Tests failed: 
*********************************************************************
Line 2, in tests/q2.py 0
Failed example:
    1 == 1
Expected:
    False
Got:
    True




To check all of the tests at once, use `otter check` without a `-q` flag:

In [37]:
!otter check scripts/file0.py

4 of 6 tests passed

Tests passed:
 q1  q3  q4  q5 


Tests failed: 
*********************************************************************
Line 2, in tests/q2.py 0
Failed example:
    1 == 1
Expected:
    False
Got:
    True




## Grading

### Notebooks

Otter uses Docker containers to execute students' submissions in parallel containers. It accepts exports from Gradescope and Canvas, or the notebook files with a JSON- or YAML-formatted metadata file. If you're using the custom metadata file, it requires an entry for each notebook, with the notebook filename as the `filename` parameter and the student identifier as `identifier`. An example of the YAML metadata can be found in `manual-test/meta.yml`.

In [10]:
with open("./manual-test/meta.yml") as f:
    for l in f.readlines()[0:10]:
        print(l[:-1])

- identifier: 0
  filename: test-nb-0.ipynb
- identifier: 1
  filename: test-nb-1.ipynb
- identifier: 2
  filename: test-nb-2.ipynb
- identifier: 3
  filename: test-nb-3.ipynb
- identifier: 4
  filename: test-nb-4.ipynb


#### Requirements

The docker container comes with the following packages installed:

* datascience
* jupyter_client
* ipykernel
* matplotlib
* pandas
* ipywidgets
* scipy
* nb2pdf
* otter-grader

If you have any other requirements besides these, create a `requirements.txt` file:

In [15]:
with open("./requirements.txt") as f:
    print(f.read())

tqdm



#### Command-Line Usage

Now let's examine the usage of the `otter` command:

In [17]:
!otter -h

usage: otter [-h] [-p NOTEBOOKS-PATH] [-t TESTS-PATH] [-o OUTPUT-PATH] [-g]
             [-c] [-j JSON] [-y YAML] [-s] [--pdf] [--filter-pdf] [-v]
             [-r REQUIREMENTS] [--containers NUM-CONTAINERS] [--image IMAGE]

optional arguments:
  -h, --help            show this help message and exit
  -p NOTEBOOKS-PATH, --path NOTEBOOKS-PATH
                        Path to directory of submissions
  -t TESTS-PATH, --tests-path TESTS-PATH
                        Path to directory of tests
  -o OUTPUT-PATH, --output-path OUTPUT-PATH
                        Path to which to write output
  -g, --gradescope      Flag for Gradescope export
  -c, --canvas          Flag for Canvas export
  -j JSON, --json JSON  Flag for path to JSON metadata
  -y YAML, --yaml YAML  Flag for path to YAML metadata
  -s, --scripts         Flag to incidicate grading Python scripts
  --pdf                 Create unfiltered PDFs for manual grading
  --filter-pdf          Create filtered PDF for manual grading
  -v, 

The `-p` flag should be the path to the directory that conains the notebooks (or the unzipped export from Canvas or Gradescope). The `g`, `c`, `y`, and `j` flags indicate the metadata type; the `y` and `j` flags require an argument that indicates the path to the metadata file. If you want to generate PDFs for manual grading, you can add the `--pdf` or `--filter-pdf` flags, which generate an unfiltered or filtered, respectively, PDF for each notebook. If you need a requirements file, pass that path to the `-r` flag. Optionally, you can set the number of docker containers or the docker image that the grader runs on using the `--containers` and `--image` flags, respectively.

Some flag defaults are listed below. If a flag is not list, it has no default value.

| Flag | Default |
|-----|-----|
| `-p` | `./` |
| `-t` | `./tests` |
| `-o` | `./` |
| `--containers` | 4 |
| `--image` | `ucbdsinfra/otter-grader` |

**If your notebooks require any other files (e.g. data files)**, place these files into the notebooks path directory (whatever the value of `-p` is). These will automatically be copied into the docker containers.

Let's run the autograder on the files in `./notebooks`. The command below tells otter that our metadata file can be found at `./notebooks/meta.yml`, that the notebooks can be found in `./notebooks`, that we have a requirements file at `./requirements.txt`, and to use verbose output.

In [28]:
!otter -y notebooks/meta.yml -p notebooks -r requirements.txt -v

Found YAML metadata...
Launching docker containers...
Launched container ea9031b991d4...
Launched container a66d5f51c747...
Launched container c0acb5938062...
Launched container cd22cc716f86...
Installing requirements in container ea9031b991d4...
Installing requirements in container a66d5f51c747...
Installing requirements in container cd22cc716f86...
Installing requirements in container c0acb5938062...
Grading notebooks in container ea9031b991d4...
Grading notebooks in container a66d5f51c747...
Grading notebooks in container cd22cc716f86...
Grading notebooks in container c0acb5938062...
Copying grades from container a66d5f51c747...
Copying grades from container ea9031b991d4...
Copying grades from container c0acb5938062...
Stopping container a66d5f51c747...
Stopping container ea9031b991d4...
Copying grades from container cd22cc716f86...
Stopping container c0acb5938062...
Stopping container cd22cc716f86...
Combining grades and saving...


We should now have a CSV file with the scores and score breakdowns at `./final_grades.csv` (since we left `-o` with its default value).

In [27]:
pd.read_csv("final_grades.csv").head()

,identifier,file,q1,q2,q3,q4,q5,total,possible
0,11,test-nb-11.ipynb,1.0,0.0,1.0,0.0,0.0,2.0,5
1,2,test-nb-2.ipynb,1.0,0.0,1.0,0.0,0.0,2.0,5
2,9,test-nb-9.ipynb,1.0,0.0,1.0,0.0,0.0,2.0,5
3,52,test-nb-52.ipynb,1.0,0.0,1.0,0.0,0.0,2.0,5
4,55,test-nb-55.ipynb,1.0,0.0,1.0,0.0,0.0,2.0,5


If the autograder ran correctly, all of the submissions should receive a 2/5, since three of the tests are written to fail on these notebooks.

#### nb2pdf

When exporting notebook PDFs, otter uses the library nb2pdf, which in turn relies on nbpdfexporter. This requires that chromium be installed on the docker container (which is true if using `ucbdsinfra/otter-grader`, the default image) but it also must be installed on the JupyterHub distribution that students use if they are going to be exporting PDFs themselves.

When generating the PDF (if filtering is turned on, either by use of the `--filter-pdf` flag or `filtering=True` in `Notebook.export`), the following cells will be included by default:

* Markdown cells
* Code cells with images in the output
* All cells tagged with `include`

If a cell falls within one of the 3 categories listed above but you do not want to include it in the exported PDF, tag the cell with `ignore`. This cell will then not be added to the manual graded PDF.

### Scripts

Otter can also grade Python scripts using the addition of the `-s` flag. It uses the same metadata and path structure as with notebooks, so the only change to our call above will be to change the path to `./scripts`. *This will overwrite `final_grades.csv` from above since we aren't changing the output path.*

In [30]:
!otter -sy scripts/meta.yml -p scripts -r requirements.txt -v

Found YAML metadata...
Launching docker containers...
Launched container 44337f32f86d...
Launched container a2079bc0d4e5...
Launched container d73dd8be994f...
Launched container 31c40d6c813e...
Installing requirements in container 44337f32f86d...
Installing requirements in container a2079bc0d4e5...
Installing requirements in container 31c40d6c813e...
Installing requirements in container d73dd8be994f...
Grading scripts in container 44337f32f86d...
Grading scripts in container a2079bc0d4e5...
Grading scripts in container d73dd8be994f...
Grading scripts in container 31c40d6c813e...
Copying grades from container 44337f32f86d...
Copying grades from container a2079bc0d4e5...
Stopping container 44337f32f86d...
Copying grades from container 31c40d6c813e...
Stopping container a2079bc0d4e5...
Copying grades from container d73dd8be994f...
Stopping container 31c40d6c813e...
Stopping container d73dd8be994f...
Combining grades and saving...


Now, we should be able to read in the grades from the scripts. If all went well, they should each have a 4/5.

In [31]:
pd.read_csv("final_grades.csv").head()

,identifier,file,q1,q2,q3,q4,q5,total,possible
0,12,file12.py,1.0,0.0,1.0,1.0,1.0,4.0,5
1,84,file84.py,1.0,0.0,1.0,1.0,1.0,4.0,5
2,78,file78.py,1.0,0.0,1.0,1.0,1.0,4.0,5
3,71,file71.py,1.0,0.0,1.0,1.0,1.0,4.0,5
4,28,file28.py,1.0,0.0,1.0,1.0,1.0,4.0,5


## Gradescope

Otter is compatible with the Gradescope autograder, and has a command line tool to generate the zipfile that is used to configure the Docker container that Gradescope's autograder usage. The base command for this utility is `otter gen`, and its help entry is given below.

In [38]:
!otter gen -h

usage: otter [-h] [-t [TESTS-PATH]] [-o [OUTPUT-PATH]] [-r [REQUIREMENTS]]
             [files [files ...]]

Generates zipfile to configure Gradescope autograder

positional arguments:
  files                 Other support files needed for grading (e.g. .py
                        files, data files)

optional arguments:
  -h, --help            show this help message and exit
  -t [TESTS-PATH], --tests-path [TESTS-PATH]
                        Path to test files
  -o [OUTPUT-PATH], --output-path [OUTPUT-PATH]
                        Path to which to write zipfile
  -r [REQUIREMENTS], --requirements [REQUIREMENTS]
                        Path to requirements.txt file


The `otter gen` command creates a zipfile at `OUTPUT-PATH/autograder.zip` which contains the necessary files for the Gradescope autograder:

* `run_autograder`: the script that executes otter
* `setup.sh`: a file that instructs Ubuntu on how to install dependencies
* `requirements.txt`: Python's list of necessary dependencies
* `tests`: the folder of test cases
* `files`: a folder containing any files needed for the notebooks to execute (e.g. data files)

The requirements file create automatically includes the otter dependencies (see [Requirements](#Requirements)), but you can optionally include your own, other ones by passing a filepath to the `-r` flag. Any files included that are not passed to a flag are automatically placed into the `files` folder in the zipfile and will be copied into the notebook directory in the Gradescope container.

Let's generate a zipfile from this directory with the tests in `./tests`, the requirements in `./requirements.txt`, and `test-df.csv` as a data file.

In [39]:
!otter gen -r requirements.txt test-df.csv

Now, we have a file `./autograder.zip` in the working directory:

In [42]:
"autograder.zip" in os.listdir()

True

If we unzip the file, you'll see all of the files listed above as well as the CSV file we added in the `autograder/files` directory:

In [61]:
unzipped = subprocess.run(["unzip", "autograder.zip", "-d", "autograder"], stdout=PIPE, stderr=PIPE)
print(unzipped.stdout.decode("utf-8"))

Archive:  autograder.zip
  inflating: autograder/run_autograder  
  inflating: autograder/setup.sh     
  inflating: autograder/requirements.txt  
   creating: autograder/tests/
  inflating: autograder/tests/q5.py  
  inflating: autograder/tests/q1.py  
  inflating: autograder/tests/q4.py  
  inflating: autograder/tests/q3.py  
  inflating: autograder/tests/q2.py  
   creating: autograder/files/
 extracting: autograder/files/test-df.csv  



In [63]:
os.listdir("autograder")

['setup.sh', 'requirements.txt', 'run_autograder', 'tests', 'files']

#### Relative Imports on Gradescope

Because of the way that the Gradescope autograder is set up, our script must change relative import syntax in order to ensure that the necessary files are loaded. **Because we cannot differentiate between package imports and relative imports, `otter gen` automatically assumes that if you are importing a .py file that it is called `utils.py`.** The regex used to change the import syntax will _only_ change the syntax if you're using the import statment

```python
from utils import *
```

For this reason, if you have any functions you need in a file that is going to be imported, _make sure that it is called `utils.py`._